In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from ipywidgets import interact

# Function Definitions

In [3]:
def load_load_data():
    load_history_raw = pd.read_csv("Load_history.csv")
    
    # step 1: prep the data for unpivoting

    # init empty df to hold prepped data
    load_data_wide = pd.DataFrame()
    
    # copy over the zone_id column (it doesn't need any prepping)
    load_data_wide['zone_id'] = load_history_raw['zone_id']
    
    # convert year, month, day columns to one date column
    load_data_wide['date'] = pd.to_datetime(load_history_raw[['year', 'month', 'day']])
    
    # bring in the hour columns
    hour_columns = [f'h{i}' for i in range(1, 25)]
    for col in hour_columns:
        load_data_wide[col] = load_history_raw[col]
    
    # step 2: unpivoting
    load_data_long = load_data_wide.melt(
        id_vars = ['zone_id', 'date'], # cols to preserve
        value_vars=hour_columns,       # cols to unpivot
        var_name='hour',               # colname for new index col (hour)
        value_name='load'              # colname for new value col (load)
    )
    
    # step 3: clean up after unpivot
    
    # convert 'hour' from string (i.e. 'h1') to numeric (i.e. 1)
    load_data_long['hour'] = load_data_long['hour'].str.extract(r'(\d+)').astype(int)
    
    # create a full datetime column by augmenting 'date' with 'hour'
    load_data_long['datetime'] = load_data_long['date'] + pd.to_timedelta(load_data_long['hour'] - 1, unit='h')
    
    # drop extra columns
    load_data_long = load_data_long[['zone_id', 'datetime', 'load']]
    
    return load_data_long

In [11]:
def load_temp_data():
    temp_history_raw = pd.read_csv("temperature_history.csv")
    
    # step 1: prep the data for unpivoting

    # init empty df to hold prepped data
    temp_data_wide = pd.DataFrame()
    
    # copy over the zone_id column (it doesn't need any prepping)
    temp_data_wide['station_id'] = temp_history_raw['station_id']
    
    # convert year, month, day columns to one date column
    temp_data_wide['date'] = pd.to_datetime(temp_history_raw[['year', 'month', 'day']])
    
    # bring in the hour columns
    hour_columns = [f'h{i}' for i in range(1, 25)]
    for col in hour_columns:
        temp_data_wide[col] = temp_history_raw[col]
    
    # step 2: unpivoting
    temp_data_long = temp_data_wide.melt(
        id_vars = ['station_id', 'date'], # cols to preserve
        value_vars=hour_columns,          # cols to unpivot
        var_name='hour',                  # colname for new index col (hour)
        value_name='temp'                 # colname for new value col (temp)
    )
    
    # step 3: clean up after unpivot
    
    # convert 'hour' from string (i.e. 'h1') to numeric (i.e. 1)
    temp_data_long['hour'] = temp_data_long['hour'].str.extract(r'(\d+)').astype(int)
    
    # create a full datetime column by augmenting 'date' with 'hour'
    temp_data_long['datetime'] = temp_data_long['date'] + pd.to_timedelta(temp_data_long['hour'] - 1, unit='h')
    
    # drop extra columns
    temp_data_long = temp_data_long[['station_id', 'datetime', 'temp']]
    
    return temp_data_long

# Analysis

In [12]:
load_data = load_load_data()
temp_data = load_temp_data()

In [13]:
load_data

,zone_id,datetime,load
0,1,2004-01-01 00:00:00,"16,853"
1,1,2004-01-02 00:00:00,"14,155"
2,1,2004-01-03 00:00:00,"14,439"
3,1,2004-01-04 00:00:00,"11,273"
4,1,2004-01-05 00:00:00,"10,750"
...,...,...,...
791995,20,2008-07-03 23:00:00,NaN
791996,20,2008-07-04 23:00:00,NaN
791997,20,2008-07-05 23:00:00,NaN
791998,20,2008-07-06 23:00:00,NaN


In [14]:
temp_data

,station_id,datetime,temp
0,1,2004-01-01 00:00:00,46.0
1,1,2004-01-02 00:00:00,43.0
2,1,2004-01-03 00:00:00,45.0
3,1,2004-01-04 00:00:00,63.0
4,1,2004-01-05 00:00:00,64.0
...,...,...,...
433747,11,2008-06-26 23:00:00,72.0
433748,11,2008-06-27 23:00:00,71.0
433749,11,2008-06-28 23:00:00,71.0
433750,11,2008-06-29 23:00:00,70.0
